In [1]:
import pandas as pd
import boto3
import json
import io
import os

# S3 Connection

In [2]:
with open("aws_credential.txt", 'r') as file:
    aws_credential=json.load(file)
s3=boto3.Session(
    profile_name = None, 
    region_name = 'us-east-2').client(
    's3',
    aws_access_key_id=aws_credential['access_key'],
    aws_secret_access_key=aws_credential['secret_key'])

In [3]:
content_list=s3.list_objects(Bucket = aws_credential['bucket'], Prefix ='visa_output/')['Contents']
# if size = 0, it's the folder resource/ itself
[d['Key'] for d in content_list if d['Size']>0] 

['visa_output/country_code_matches.csv',
 'visa_output/country_list.txt',
 'visa_output/df_iv.csv',
 'visa_output/df_niv.csv',
 'visa_output/iv_alltime.csv',
 'visa_output/niv_alltime.csv',
 'visa_output/time_iv.txt',
 'visa_output/time_niv.txt']

In [4]:
country_codes_bytes=s3.get_object(Bucket = aws_credential['bucket'], 
              Key = 'visa_output/country_code_matches.csv')['Body'].read()

In [5]:
# read-in well-built .csv files, set delimiter to ",", and then the first row as header
# set data type of country codes to string object, since none start w/ 0 so no need to zfill()
country_codes=pd.read_csv(io.BytesIO(country_codes_bytes), 
                          delimiter = ",", 
                          header = 0, 
                          dtype={'code':str})

In [6]:
country_codes.head(3)

,query,match,code,iso
0,AFGHANISTAN,Afghanistan,5310,AF
1,ALBANIA,Albania,4810,AL
2,ALGERIA,Algeria,7210,DZ


df_iv_bytes = s3.get_object(Bucket=aws_credential['bucket'],
                           Key='visa_output/df_iv.csv')['Body'].read()
df_niv_bytes = s3.get_object(Bucket=aws_credential['bucket'],
                           Key='visa_output/df_niv.csv')['Body'].read()
df_iv=pd.read_csv(io.BytesIO(df_iv_bytes), decimal=",", header=0)
df_niv=pd.read_csv(io.BytesIO(df_niv_bytes), decimal=",", header=0)

In [8]:
df_iv.head(3)

,nationality,visa,count,time
0,AFGHANISTAN,CR1,11,2017-03-31
1,AFGHANISTAN,DV1,2,2017-03-31
2,AFGHANISTAN,DV2,1,2017-03-31


In [9]:
df_niv.head(3)

,nationality,visa,count,time
0,NON NATIONALITY BASED ISSUANCES,A2,1,2017-03-31
1,NON NATIONALITY BASED ISSUANCES,B1,1,2017-03-31
2,NON NATIONALITY BASED ISSUANCES,B1/B2,163,2017-03-31


# Apply Labels